In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
from tqdm import tqdm

# Any results you write to the current directory are saved as output.

['OutputRecommendation1.txt']


In [2]:
df = pd.read_csv('../input/OutputRecommendation1.txt')
df.head()

,Id,ASIN,title,group,salesrank,copurchased,categories,totalReviews,avgRating
0,1,0827229534,Patterns of Preaching,Book,396585,0804215715 156101074X 0687023955 0687074231 08...,Books Subjects Religion & Spirituality Christi...,2,5.0
1,2,0738700797,Candlemas,Book,168596,0738700827 1567184960 1567182836 0738700525 07...,Books Subjects Religion & Spirituality Earth-B...,12,4.5
2,4,0842328327,Life Application Bible Commentary,Book,631289,0842328130 0830818138 0842330313 0842328610 08...,Books Subjects Religion & Spirituality Christi...,1,4.0
3,5,1577943082,Prayers That Avail Much for Business,Book,455160,157794349X 0892749504 1577941829 0892749563 15...,Books Subjects Religion & Spirituality Christi...,0,0.0
4,6,0486220125,How the Other Half Lives,Book,188784,0486401960 0452283612 0486229076 0714840343 03...,Books Subjects Arts & Photography Photography ...,17,4.0


In [3]:
dfDVD = df.loc[ df['group'] == 'DVD']
dfDVD.head()


,Id,ASIN,title,group,salesrank,copurchased,categories,totalReviews,avgRating
16,21,0790747324,The Time Machine,DVD,795,B00007JMD8 6305350221 B00004RF9B B00005JKFR B0...,DVD Genres Drama General,140,4.5
29,37,B000056PNC,Mark Messier - Leader Champion & Legend,DVD,46018,B00020W0WO B0000C8AO7 B0002WZRUO B0000C8AO5 B0...,DVD Genres Sports General,7,3.5
32,42,B000056PNB,Pixote,DVD,19685,B0002LJTIG B00004TJRE B0001K2S36 B0000DIJON B0...,DVD Genres Art House & International General,24,4.5
34,44,B000056PNE,The NBA's 100 Greatest Plays,DVD,33124,B0002IKSFC B0000C8AO1 B0001WTVU2 B00013F38U B0...,DVD Genres Sports General,13,4.0
108,149,B00000IC80,Pot O' Gold/Made for Each Other,DVD,51310,B00006AUGQ B00005RERR B0002KPHZ6,DVD Genres Comedy General,0,0.0


In [4]:
newDf = pd.DataFrame(columns=['ASINs', 'TitleSimilarity', 'CategorySimilarity', 'SalesRank', 'CoPurchasedProducts'])
newDf

,ASINs,TitleSimilarity,CategorySimilarity,SalesRank,CoPurchasedProducts


In [5]:
for i in range(int(dfDVD.shape[0]/4)):
    row = dfDVD.iloc[i][:]
    rowASIN = row['ASIN']
    cat_i = row['categories'].split(' ')
    title_i = row['title'].split(' ')
    #copurchasedASIN = row['copurchased'].split(' ')
    for coASIN in dfDVD.iloc[i]['copurchased'].split(' '):
        cat_jaccard_similarity = 0
        title_jaccard_similarity = 0
        cat_j_row = dfDVD.loc[ dfDVD['ASIN'] == coASIN ]
        cat_j = cat_j_row['categories'].to_string().split(' ')[1:]
#         if i<2:
#             print(coASIN)
#             print(cat_j)
        cat_iUcat_j = set(cat_i).union(set(cat_j))
        cat_iIcat_j = set(cat_i).intersection(set(cat_j))
        if len(cat_iUcat_j)>0:
            cat_jaccard_similarity = round( len(cat_iIcat_j)/len(cat_iUcat_j) , 2)
        else:
            cat_jaccard_similarity = 0
        
        title_j = cat_j_row['title'].to_string().split(' ')
        title_iUtitle_j = set(title_i).union(set(title_j))
        title_iItitle_j = set(title_i).intersection(set(title_j))
        if len(title_iUtitle_j)>0:
            title_jaccard_similarity = round( len(title_iItitle_j)/ len(title_iUtitle_j) , 2)
        else:
            title_jaccard_similarity = 0
        l = int(row['salesrank'])
        #print(cat_j_row['salesrank'].to_string().split(' ')[0])
        try:
            r =  int(cat_j_row['salesrank'].to_string().split(' ')[0] )
            #print(l,r)
            SalesRank = max( l, r)
            newDf = newDf.append(pd.Series([ rowASIN+' '+coASIN, title_jaccard_similarity, cat_jaccard_similarity, SalesRank, "Yes" ], index=newDf.columns ), ignore_index=True)
        except ValueError:
            pass
        

In [7]:
newDf.head()

,ASINs,TitleSimilarity,CategorySimilarity,SalesRank,CoPurchasedProducts
0,0790747324 B00007JMD8,0.00,0.44,74506,Yes
1,0790747324 6305350221,0.11,0.27,12021,Yes
2,0790747324 B00004RF9B,0.00,0.27,62320,Yes
3,0790747324 B00005JKFR,0.10,0.44,360637,Yes
4,0790747324 B00005NG6A,0.00,0.44,362669,Yes


In [9]:
newDf.shape

(12591, 5)

In [10]:
dfBook = df.loc[ df['group'] == 'Book' ]
dfBook.head()

,Id,ASIN,title,group,salesrank,copurchased,categories,totalReviews,avgRating
0,1,0827229534,Patterns of Preaching,Book,396585,0804215715 156101074X 0687023955 0687074231 08...,Books Subjects Religion & Spirituality Christi...,2,5.0
1,2,0738700797,Candlemas,Book,168596,0738700827 1567184960 1567182836 0738700525 07...,Books Subjects Religion & Spirituality Earth-B...,12,4.5
2,4,0842328327,Life Application Bible Commentary,Book,631289,0842328130 0830818138 0842330313 0842328610 08...,Books Subjects Religion & Spirituality Christi...,1,4.0
3,5,1577943082,Prayers That Avail Much for Business,Book,455160,157794349X 0892749504 1577941829 0892749563 15...,Books Subjects Religion & Spirituality Christi...,0,0.0
4,6,0486220125,How the Other Half Lives,Book,188784,0486401960 0452283612 0486229076 0714840343 03...,Books Subjects Arts & Photography Photography ...,17,4.0


In [11]:
for i in tqdm(range(int(dfBook.shape[0]/25))):
    bookASIN = dfBook.iloc[i]['ASIN']
    dvdASIN = newDf.iloc[i]['ASINs'].split(' ')[0]
    cat_i = dfBook.iloc[i]['categories'].split(' ')
    title_i = dfBook.iloc[i]['title'].split(' ')
    
    cat_jaccard_similarity = 0
    title_jaccard_similarity = 0
    #cat_j_row = dfDVD.loc[ dfDVD['ASIN'] == coASIN ]
    cat_j_row = dfBook.iloc[i][:]
    cat_j = cat_j_row['categories'].split(' ')[1:]

    cat_iUcat_j = set(cat_i).union(set(cat_j))
    cat_iIcat_j = set(cat_i).intersection(set(cat_j))
    if len(cat_iUcat_j)>0:
        cat_jaccard_similarity = round( len(cat_iIcat_j)/len(cat_iUcat_j) , 2)
    else:
        cat_jaccard_similarity = 0

    title_j = cat_j_row['title'].split(' ')
    title_iUtitle_j = set(title_i).union(set(title_j))
    title_iItitle_j = set(title_i).intersection(set(title_j))
    if len(title_iUtitle_j)>0:
        title_jaccard_similarity = round( len(title_iItitle_j)/ len(title_iUtitle_j) , 2)
    else:
        title_jaccard_similarity = 0
    l = int(row['salesrank'])
    #print(cat_j_row['salesrank'])
    try:
        r =  int(cat_j_row['salesrank'] )
        #print(l,r)
        SalesRank = max( l, r)
        newDf = newDf.append(pd.Series([ rowASIN+' '+coASIN, title_jaccard_similarity, cat_jaccard_similarity, SalesRank, "No" ], index=newDf.columns ), ignore_index=True)
    except ValueError:
        pass

100%|██████████| 10659/10659 [01:27<00:00, 122.00it/s]


In [12]:
newDf.shape

(23250, 5)

In [13]:
Y = newDf.pop('CoPurchasedProducts')
X = newDf
X.head()

,ASINs,TitleSimilarity,CategorySimilarity,SalesRank
0,0790747324 B00007JMD8,0.00,0.44,74506
1,0790747324 6305350221,0.11,0.27,12021
2,0790747324 B00004RF9B,0.00,0.27,62320
3,0790747324 B00005JKFR,0.10,0.44,360637
4,0790747324 B00005NG6A,0.00,0.44,362669


In [15]:
Y.head()

0    Yes
1    Yes
2    Yes
3    Yes
4    Yes
Name: CoPurchasedProducts, dtype: object

In [16]:
X1 = newDf.pop('ASINs')
X2 = newDf
X2.head()

,TitleSimilarity,CategorySimilarity,SalesRank
0,0.00,0.44,74506
1,0.11,0.27,12021
2,0.00,0.27,62320
3,0.10,0.44,360637
4,0.00,0.44,362669


In [17]:
Y_onehot = Y.copy()
Y_onehot = pd.get_dummies(Y_onehot, columns=['CoPurchasedProducts'], prefix ='CoPurchasedProducts')

print(Y_onehot.head())

   CoPurchasedProducts_No  CoPurchasedProducts_Yes
0                       0                        1
1                       0                        1
2                       0                        1
3                       0                        1
4                       0                        1


In [18]:
Y_onehot.pop('CoPurchasedProducts_Yes')

0        1
1        1
2        1
3        1
4        1
5        1
6        1
7        1
8        1
9        1
10       1
11       1
12       1
13       1
14       1
15       1
16       1
17       1
18       1
19       1
20       1
21       1
22       1
23       1
24       1
25       1
26       1
27       1
28       1
29       1
        ..
23220    0
23221    0
23222    0
23223    0
23224    0
23225    0
23226    0
23227    0
23228    0
23229    0
23230    0
23231    0
23232    0
23233    0
23234    0
23235    0
23236    0
23237    0
23238    0
23239    0
23240    0
23241    0
23242    0
23243    0
23244    0
23245    0
23246    0
23247    0
23248    0
23249    0
Name: CoPurchasedProducts_Yes, Length: 23250, dtype: uint8

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X2, Y_onehot, test_size=0.4, random_state=42)

In [42]:
from sklearn.svm import SVC
model = SVC(C=1000, kernel='rbf', degree=2, gamma='auto', shrinking=True, tol=0.0001, cache_size=200, verbose=1, max_iter=-1, decision_function_shape='ovr', random_state=42)

In [43]:
model.fit(X_train,y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LibSVM]

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=2, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.0001, verbose=1)

In [44]:
y_pred = model.predict(X_test)

In [46]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_pred,y_test)
acc

0.8303225806451613